In [6]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.manifold import TSNE


import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('drive/My Drive/glove.6B.200d.txt') 

In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [375]:
### CHecking the working of word_to_index and index_to_word
word = "postgraduate"
index = 228777
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

the index of postgraduate in the vocabulary is 289695
the 228777th word in the vocabulary is m.com


In [0]:
### Finally not used
def sentence_to_avg(sentence, word_to_vec_map):
  
    words = sentence.lower().split()
    avg = np.zeros(np.size(word_to_vec_map[words[0]]))
    for w in words:
        avg += word_to_vec_map[w]
    avg = avg/len(words)
    return avg

In [22]:
### Checking above piece of code
avg = sentence_to_avg("UG", word_to_vec_map)
print("avg = ", avg)
len(avg)

avg =  [-0.04734   -0.061418   0.28016    0.25011    0.66352   -0.25791
  0.29937   -0.24376   -0.64111    0.45021   -0.54224   -0.28894
 -0.18066   -0.054409  -0.20085    0.077443   0.064688   0.0384
  0.51145    0.1404     0.5524    -0.21728    0.25235   -0.50984
  0.47499   -0.47774   -0.27016   -0.27463   -0.42415   -0.63635
 -0.037956   0.3707    -0.040264  -0.48404    0.35803   -0.0047529
  0.48459   -0.086105   0.084618  -0.055723   0.10765   -0.1866
  0.1614    -0.47866    0.53266   -0.018869   0.17879   -0.0034171
  0.65881   -0.49567   -0.082975  -0.017535   0.32772    0.38446
 -0.18644    0.37991   -0.22289    0.22942    0.16312    0.13109
  0.42933   -0.17329    0.11604    0.68096   -0.18796    0.073183
  0.0092278  0.066847   0.026209  -0.03482   -0.40936    0.04531
 -0.1322    -0.65706   -0.12577   -0.30149    0.46237   -0.55123
  0.2612    -0.18326   -0.065977   0.43944    0.14724    0.18109
 -0.19574   -0.4993    -0.29004   -0.033837  -0.46409    0.36511
  0.54648   -0.

200

In [0]:
### returns the sentences to indices conversion
def sentences_to_indices(X, word_to_index, max_len):
    
    m = X.shape[0]                                   # number of training examples
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        sentence_words =X[i].lower().split()
        j = 0
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w]
            j = j+1
            
    
    return X_indices

In [19]:
#### Checking sentences_to_indices function
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 30)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[155345. 225122.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.]
 [220930. 286375.  69714.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.]]


In [0]:
### Embedding layer output is returned
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]     
    emb_matrix = np.zeros((vocab_len,emb_dim))
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
    embedding_layer = Embedding(input_dim = vocab_len, output_dim = emb_dim,trainable = False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [0]:
from keras.layers import Bidirectional, Sequential
def classify(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)   
    X = LSTM(32, return_sequences=True)(embeddings)
    #X = Dropout(0.5)(X)
    X = LSTM(32, return_sequences=False)(X)
    #X = Dropout(0.5)(X)
    X = Dense(10)(X)
    X = Activation('softmax')(X)

    model = Model(inputs=sentence_indices, outputs=X)
    return model

In [396]:
model = classify((10,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 10)                0         
_________________________________________________________________
embedding_18 (Embedding)     (None, 10, 200)           80000200  
_________________________________________________________________
lstm_30 (LSTM)               (None, 10, 32)            29824     
_________________________________________________________________
lstm_31 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_15 (Dense)             (None, 10)                330       
_________________________________________________________________
activation_15 (Activation)   (None, 10)                0         
Total params: 80,038,674
Trainable params: 38,474
Non-trainable params: 80,000,200
___________________________________________________________

In [0]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
import pandas
import re
from operator import itemgetter
df = pandas.read_csv('drive/My Drive/naukri_com-job_sample.csv')
Y = {}
Y_num = {}
top_Y = {}
X = []
pred = []
ind = 0
# print(df['education'][0].lower()) 

for i in range(21999) :
  y = df['industry'][i]
  if(type(y) == str) :
    if y not in Y :
      Y[y] = ind
      Y_num[y] = 1
      ind += 1
    else :
      Y_num[y] += 1 


so_li = sorted(Y_num.items() , key = itemgetter(1))
ind = 0
for i in range(10) :
  top_Y[so_li[-i-1][0]] = ind
  ind+=1
  
  
for i in range(21999) :
  y = df['industry'][i]
  s = df['education'][i]
  if(type(s) == str and type(y) == str and y in top_Y) :
    s = s.lower()
    s = s.replace('ug:' , 'undergraduate').replace('pg:' , 'postgraduate ').replace('doctorate:' , 'doctorate ').replace('(',' ').replace(')', ' ').replace('bhm','').replace('b.pharma', 'pharma').replace('b.b.a', 'business').replace('b.m.s','management').replace('gyneocology','gynaecology').replace('obstretrics','obstetrics').replace('opthalmology','ophthalmology').replace('m.pharma', 'pharma').replace('mvsc','Veterinary').replace('b.des.','design').replace('arts&humanities','arts humanities').replace('any specialization','').replace('any graduate','').replace('any postgraduate','')
    k = s.replace(',',' ').replace('-',' ').replace('/' , ' ').split()
    #k = list(set(k))
    if len(k) <= 30 :
      X.append(" ".join(k))
      pred.append(top_Y[y])

      
pred = np.array(pred)

In [319]:
print(len(pred))
print(len(X))
print(pred[0:100])
print(top_Y)

15094
15094
[0 3 0 0 0 0 0 0 0 0 0 4 0 0 0 0 3 3 5 0 0 2 0 6 0 0 0 0 0 0 0 0 0 3 0 0 5
 0 0 0 5 4 5 2 0 3 1 0 0 8 0 0 4 0 5 0 0 4 3 4 0 4 4 0 0 4 0 0 6 1 0 0 0 3
 0 0 4 0 0 6 0 4 7 2 1 3 1 0 4 9 3 1 2 4 2 7 0 0 1 3]
{'IT-Software / Software Services': 0, 'Education / Teaching / Training': 1, 'BPO / Call Centre / ITES': 2, 'Banking / Financial Services / Broking': 3, 'Recruitment / Staffing': 4, 'Internet / Ecommerce': 5, 'Pharma / Biotech / Clinical Research': 6, 'Medical / Healthcare / Hospitals': 7, 'Automobile / Auto Anciliary / Auto Components': 8, 'Construction / Engineering / Cement / Metals': 9}


In [0]:
X_ = np.array(X[0:10000])
X_train_indices = sentences_to_indices(X_, wor1d_to_index, 30)
Y_train_oh = convert_to_one_hot(np.array(pred[0:10000]), C = 10)

In [0]:
model.fit(X_train_indices, Y_train_oh, epochs = 10, batch_size = 32, shuffle=True)

In [0]:
from keras.models import load_model
model.save('drive/My Drive/my_model.h5')
model = load_model('drive/My Drive/my_model.h5')

In [0]:
def output(s,model,dictt):
  s = sentences_to_indices(s, word_to_index, 30)
  out = model.predict(s)
  print(out)
  return dictt[np.argmax(out)]

In [30]:
rev_y = dict([(top_Y[k],k) for k in top_Y])
rev_y

{0: 'IT-Software / Software Services',
 1: 'Education / Teaching / Training',
 2: 'BPO / Call Centre / ITES',
 3: 'Banking / Financial Services / Broking',
 4: 'Recruitment / Staffing',
 5: 'Internet / Ecommerce',
 6: 'Pharma / Biotech / Clinical Research',
 7: 'Medical / Healthcare / Hospitals',
 8: 'Automobile / Auto Anciliary / Auto Components',
 9: 'Construction / Engineering / Cement / Metals'}

In [0]:
import csv
with open('drive/My Drive/dictionary2.csv', 'w') as f:
    for key in rev_y:
        f.write("%s,%s\n"%(key , rev_y[key]))

In [34]:
from keras.models import load_model
model = load_model('drive/My Drive/my_model2.h5')
X_train_indices = sentences_to_indices(np.array(['undergraduate b.tech mechanical']), word_to_index, 10)
predictions = model.predict(X_train_indices)
index = np.argmax(predictions)
print(rev_y[index])

### THe down is the output

Automobile / Auto Anciliary / Auto Components


In [26]:
### Index value of given input
index

8